<a href="https://colab.research.google.com/github/SaketMunda/introduction-to-nlp/blob/master/skimlit_nlp_milestone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone Project : SkimLit 🗒 🔥

In this project, we're going to be putting what we've learned during the `introduction_to_nlp` notebook.

We're going to be replicating the deep learning model behind the 2017 paper [PubMed 200K RCT : a Dataset for Sequential Sentence Classification in Medical Abstracts](https://arxiv.org/abs/1710.06071)

RCT : Randomized Controlled Trial abstracts

The goal of the dataset was to explore the ability for NLP models to classify sentences which appear in sequential order.

Input : an abstract of a RCT

Output : What role does each sentence serve in the abstract ?

**Problem**:

The number of RCT papers released is continuing to increase, those without structured abstracts can be hard to read and in turn slow down researchers moving through the literature.

**Solution**:

Create an NLP model to classify abstract sentences into the role they play (e.g objective, methods, results, etc) to enable researchers to skim through the literature (hence SkimLit) and dive deeper when necessary.

**Dataset**:

[PubMed RCT200k from Github](https://github.com/Franck-Dernoncourt/pubmed-rct)


**Model(Insipiration)**:

[Neural Networks for joint sentence classification in medical paper abstracts](https://arxiv.org/pdf/1612.05251.pdf)



In [ ]:
# check the gpu
!nvidia-smi -L

## Get the Data

In [ ]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct
!ls pubmed-rct